<a href="https://colab.research.google.com/github/BCI-and-Neuroergonomics-Lab/3D-MB-CNN/blob/dev/BCICIV_2a_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Drive

In [ ]:
# Loading your drive so CoLab has access to it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
import os, sys, re
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import numpy as np
from glob import glob
import warnings
try:
  import mne
except ModuleNotFoundError:
  !pip install mne
  import mne

     |████████████████████████████████| 7.5 MB 6.5 MB/s 


# Handle Warnings

In [ ]:
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

Constants


In [ ]:
# Data Processing Mode ('3D' or '2D')

MODE = '3D'

# File Patterns for Data Files

DATA_FILE_PATTERN = "/A0{}{}.gdf"
KEY_FILE_PATTERN = "/A0{}{}.mat"
SUFFIX_3D_DIRS = "_3d/"
SUFFIX_2D_DIRS = "_2d/"

# Data Directories
DEFAULT_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_raw"

KEY_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_raw/true_labels"

PROCESSED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_processed"

CROPPED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_cropped"

AVERAGED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_averaged"

SHUFFLED_DIR = "/content/drive/MyDrive/BCI-Lab-Drive/Datasets/BCICIV_2a/BCICIV_2a_shuffled"

# Data Split

TRAIN_DIR_APPEND = "train/"

VAL_DIR_APPEND = "val/"

TEST_DIR_APPEND = "test/"

SPLIT_TYPES = ('train', 'test', 'val')
SPLIT_DIR_DICT = {'train': TRAIN_DIR_APPEND, 'test': TEST_DIR_APPEND, 'val': VAL_DIR_APPEND}

# Split Ratios, results in a 60/20/20 split
# First data is split between training and val/test
# Second val/test is split into val and test
TRAIN_SPLIT = 0.6
TEST_SPLIT = 0.5

SHUFFLE_SPLITS = True

# Data Processing Parameters

# Event Tags
# '783' corresponds to the eval set events, since they are not keyed
EVENT_TAGS = ('769', '770', '771', '772', '783')

# Event Tag Meanings
# EVENT_DICT = {'769': 'Left', '770': 'Right', '771': 'Foot', '772': 'Tongue'}
EVENT_DICT = {'769': 0, '770': 1, '771': 2, '772': 3, '783': 4}

BADS = ['EOG-left', 'EOG-central', 'EOG-right']

NUM_CHANNELS = 22

T_AFTER_STIM = 0.5

# Initial time window extracted from raw data
WINDOW = 2
# Data frequency
FREQ = 250
# Lower Band of Filter
LOW_BAND_FREQ = 4
# Higher Band of Filter
HIGH_BAND_FREQ = 40
# Number of classes
NUM_CLASSES = 4
# Number of subjects
NUM_SUBJECTS = 9
# X dim of 3D data
X_DIM_3D = 7
# Y dim of 3D data
Y_DIM_3D = 6
# Window size post cropping
WINDOW_SIZE = 313
# Step size while cropping
STEP_SIZE = 20
# Which channels are included, matched with BCICIV_2a dataset channels
INCLUDE = ('EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16')


Data Processing Class

In [ ]:
class DataProcessing:
  def __init__(self, data_dir=DEFAULT_DIR, event_tags = EVENT_TAGS, mode='3D'):
    if (mode != '3D' and mode != '2D'):
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    self.dim = mode
    self.event_tags = event_tags
    self.data_files = {}
    self.data_file_counts = {}
    train_count = 0
    eval_count = 0
    for subject in range(1, NUM_SUBJECTS+1):
      self.data_files[subject] = {}
      for train_or_eval in 'TE':
        data_file_name = data_dir + DATA_FILE_PATTERN.format(subject, train_or_eval)
        if (os.path.exists(data_file_name)):
          self.data_files[subject][train_or_eval] = data_file_name
          if (train_or_eval == 'T'):
            train_count += 1
          if (train_or_eval == 'E'):
            eval_count += 1
        else:
          raise Exception("Failed to find data file, verify files are located in data directory.")
    print("Found {} Training Files".format(train_count))
    print("Found {} Evaluation Files".format(eval_count))

  def __convertIndices(self, channel):
    xDict = {'EEG-Fz':3, 'EEG-0':1, 'EEG-1':2, 'EEG-2':3, 'EEG-3':4, 'EEG-4':5, 'EEG-5':0, 'EEG-C3':1, 'EEG-6':2, 'EEG-Cz':3, 'EEG-7':4, 'EEG-C4':5, 'EEG-8':6, 'EEG-9':1, 'EEG-10':2, 'EEG-11':3, 'EEG-12':4, 'EEG-13':5, 'EEG-14':2, 'EEG-Pz':3, 'EEG-15':4, 'EEG-16':3}
    yDict = {'EEG-Fz':0, 'EEG-0':1, 'EEG-1':1, 'EEG-2':1, 'EEG-3':1, 'EEG-4':1, 'EEG-5':2, 'EEG-C3':2, 'EEG-6':2, 'EEG-Cz':2, 'EEG-7':2, 'EEG-C4':2, 'EEG-8':2, 'EEG-9':3, 'EEG-10':3, 'EEG-11':3, 'EEG-12':3, 'EEG-13':3, 'EEG-14':4, 'EEG-Pz':4, 'EEG-15':4, 'EEG-16':5}
    return xDict[channel], yDict[channel]

  def __liveProgress(self, tag, current, max):
    output = '\r' + tag + " - ["
    for i in range(current):
      output += '='
    for i in range(max - current):
      output += '-'
    print(output + "] - {}/{} - {:.2%}".format(current, max, current/max), end='')

  def processData(self):
    self.processed_data = {'train': {}, 'test': {}, 'val': {}}
    if (self.dim == '3D'):
      data_shape = [0, X_DIM_3D, Y_DIM_3D, int(WINDOW*FREQ)]
    elif (self.dim == '2D'):
      data_shape = [0, NUM_CHANNELS, int(WINDOW*FREQ)]
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    key_shape =[0, NUM_CLASSES]
    
    tmin = T_AFTER_STIM
    tmax = tmin + WINDOW - (1/FREQ)

    for subject in range(1, NUM_SUBJECTS+1):
      self.processed_data['train'][subject] = {}
      self.processed_data['test'][subject] = {}
      self.processed_data['val'][subject] = {}

      training_data = None
      training_keys = None
      eval_data = None
      eval_keys = None

      for num, train_or_eval in enumerate('TE'):
        self.__liveProgress("Processing Data", ((subject-1)*2)+num+1, NUM_SUBJECTS*2)

        # This code will need revisiting, but want to get it working for now
        raw = mne.io.read_raw_gdf(self.data_files[subject][train_or_eval], preload=True, verbose=False)
        raw.filter(LOW_BAND_FREQ, HIGH_BAND_FREQ)
        events, temp_id = mne.events_from_annotations(raw, verbose=False)
        event_id = {}
        conversion_dict = {}
        for key, value in temp_id.items():
          if (key in self.event_tags):
            event_id[key] = value
            conversion_dict[value] = EVENT_DICT[key]
        raw.info['bads'] = BADS
        picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
        epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False, event_repeated='merge', picks=picks, baseline=None, preload=True, verbose=False)
        num_trials = len(epochs)
        data_shape[0] = num_trials
        data = np.zeros(data_shape)
        epochs.reorder_channels(INCLUDE)
        raw_data = epochs.get_data()
        if (self.dim == '3D'):
          for (i, channel) in enumerate(INCLUDE):
            x, y = self.__convertIndices(channel)
            data[:, x, y, :] = raw_data[:, i, :]
        elif (self.dim == '2D'):
          data = raw_data
        else:
          raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
        key_shape[0] = num_trials
        keys = np.zeros(key_shape)
        file_name = KEY_DIR + KEY_FILE_PATTERN.format(subject, train_or_eval)
        key_file = loadmat(file_name)
        keys = key_file['classlabel'] - 1
        keys_1 = np.max(keys) + 1
        keys = np.eye(keys_1)[keys]
        keys = np.squeeze(keys)

        if (train_or_eval == 'T'):
          training_data = data
          training_keys = keys
        elif (train_or_eval == 'E'):
          eval_data = data
          eval_keys = keys

      all_data = np.concatenate((training_data, eval_data))
      all_keys = np.concatenate((training_keys, eval_keys))

      data_train, data_test_val, key_train, key_test_val = train_test_split(all_data, all_keys, train_size=TRAIN_SPLIT, shuffle=SHUFFLE_SPLITS, stratify=(all_keys if SHUFFLE_SPLITS else None))
      data_test, data_val, key_test, key_val = train_test_split(data_test_val, key_test_val, train_size=TEST_SPLIT, shuffle=SHUFFLE_SPLITS, stratify=(key_test_val if SHUFFLE_SPLITS else None))

      self.processed_data['train'][subject] = (data_train, key_train)
      self.processed_data['test'][subject] = (data_test, key_test)
      self.processed_data['val'][subject] = (data_val, key_val)

    print("\rFinished processing data")

  def saveProcessedData(self):
    if (self.dim == '3D'):
      fileBase = PROCESSED_DIR + SUFFIX_3D_DIRS
    elif (self.dim == '2D'):
      fileBase = PROCESSED_DIR + SUFFIX_2D_DIRS
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    for split_num, split_type in enumerate(SPLIT_TYPES):
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Saving Processed Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        np.save(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + "D_processed.npy", self.processed_data[split_type][subject][0])
        np.save(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + "K_processed.npy", self.processed_data[split_type][subject][1])
    print("\rFinished saving processed data")

  def loadProcessedData(self, mode = None):
    if (mode == None):
      mode = self.dim
    if (mode == '3D'):
      fileBase = PROCESSED_DIR + SUFFIX_3D_DIRS
    elif (mode == '2D'):
      fileBase = PROCESSED_DIR + SUFFIX_2D_DIRS
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    self.processed_data = {}
    fileSuffD = "D_processed.npy"
    fileSuffK = "K_processed.npy"
    for split_num, split_type in enumerate(SPLIT_TYPES):
      self.processed_data[split_type] = {}
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Loading Processed Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        d = np.load(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + fileSuffD)
        k = np.load(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + fileSuffK)
        self.processed_data[split_type][subject] = (d, k)
    print("\rFinished loading processed data")
  
  def cropData(self):
    steps = ((int(WINDOW*FREQ) - WINDOW_SIZE) // STEP_SIZE) + 1
    self.cropped_data = {}
    for split_num, split_type in enumerate(SPLIT_TYPES):
      self.cropped_data[split_type] = {}
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Cropping Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        if (self.dim == '3D'):
          data_shape = (int(steps*len(self.processed_data[split_type][subject][0])), X_DIM_3D, Y_DIM_3D, WINDOW_SIZE)
        elif (self.dim == '2D'):
          data_shape = (int(steps*len(self.processed_data[split_type][subject][0])), len(INCLUDE), WINDOW_SIZE)
        else:
          raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
        key_shape = (int(steps*len(self.processed_data[split_type][subject][1])), NUM_CLASSES)
        d = np.zeros(data_shape)
        for sample in range(len(self.processed_data[split_type][subject][0])):
          for step in range(steps):
            if (self.dim == '3D'):
              d[int(sample*steps + step)] = self.processed_data[split_type][subject][0][sample][:, :, (step*STEP_SIZE):(step*STEP_SIZE)+WINDOW_SIZE]
            elif (self.dim == '2D'):
              d[int(sample*steps + step)] = self.processed_data[split_type][subject][0][sample][:, (step*STEP_SIZE):(step*STEP_SIZE)+WINDOW_SIZE]
        k = np.zeros(key_shape)
        for sample in range(len(self.processed_data[split_type][subject][1])):
          for step in range(steps):
            k[int(sample*steps + step)] = self.processed_data[split_type][subject][1][sample]
        self.cropped_data[split_type][subject] = (d, k)
    print("\rFinished cropping data")

  def saveCroppedData(self):
    if (self.dim == '3D'):
      fileBase = CROPPED_DIR + SUFFIX_3D_DIRS
    elif (self.dim == '2D'):
      fileBase = CROPPED_DIR + SUFFIX_2D_DIRS
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    for split_num, split_type in enumerate(SPLIT_TYPES):
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Saving Cropped Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        np.save(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + "D_cropped.npy", self.cropped_data[split_type][subject][0])
        np.save(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + "K_cropped.npy", self.cropped_data[split_type][subject][1])
    print("\rFinished saving cropped data")
  
  def loadCroppedData(self, mode = None):
    if (mode == None):
      mode = self.dim
    if (mode == '3D'):
      fileBase = CROPPED_DIR + SUFFIX_3D_DIRS
    elif (mode == '2D'):
      fileBase = CROPPED_DIR + SUFFIX_2D_DIRS
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    self.cropped_data = {}
    fileSuffD = "D_cropped.npy"
    fileSuffK = "K_cropped.npy"
    for split_num, split_type in enumerate(SPLIT_TYPES):
      self.cropped_data[split_type] = {}
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Loading Cropped Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        d = np.load(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + fileSuffD)
        k = np.load(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + fileSuffK)
        self.cropped_data[split_type][subject] = (d, k)
    print("\rFinished loading cropped data")
  
  def averageData(self):
    self.averaged_data = {}
    if (self.dim == '3D'):
      data_shape = (X_DIM_3D, Y_DIM_3D)
    elif (self.dim == '2D'):
      data_shape = (len(INCLUDE))
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    for split_num, split_type in enumerate(SPLIT_TYPES):
      self.averaged_data[split_type] = {}
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Averaging Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        self.averaged_data[split_type][subject] = (np.zeros(self.cropped_data[split_type][subject][0].shape), self.cropped_data[split_type][subject][1])
        for sample in range(len(self.cropped_data[split_type][subject][0])):
          a = np.zeros(data_shape)
          if (self.dim == '3D'):
            for x in range(X_DIM_3D):
              for y in range(Y_DIM_3D):
                a[x, y] = np.average(self.cropped_data[split_type][subject][0][sample][x, y])
            a = np.repeat(a[:, :, np.newaxis], WINDOW_SIZE, axis=2)
          elif (self.dim == '2D'):
            for channel in range(len(INCLUDE)):
              a[channel] = np.average(self.cropped_data[split_type][subject][0][sample][channel])
            a = np.repeat(a[:, np.newaxis], WINDOW_SIZE, axis=1)
          else:   
            raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
          self.averaged_data[split_type][subject][0][sample] = self.cropped_data[split_type][subject][0][sample] - a
    print("\rFinished averaging data")

  def saveAveragedData(self):
    if (self.dim == '3D'):
      fileBase = AVERAGED_DIR + SUFFIX_3D_DIRS
    elif (self.dim == '2D'):
      fileBase = AVERAGED_DIR + SUFFIX_2D_DIRS
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    for split_num, split_type in enumerate(SPLIT_TYPES):
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Saving Averaged Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        np.save(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + "D_averaged.npy", self.averaged_data[split_type][subject][0])
        np.save(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + "K_averaged.npy", self.averaged_data[split_type][subject][1])
    print("\rFinished saving averaged data")

  def loadAveragedData(self, mode = None):
    if (mode == None):
      mode = self.dim
    if (mode == '3D'):
      fileBase = AVERAGED_DIR + SUFFIX_3D_DIRS
    elif (mode == '2D'):
      fileBase = AVERAGED_DIR + SUFFIX_2D_DIRS
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    self.averaged_data = {}
    fileSuffD = "D_averaged.npy"
    fileSuffK = "K_averaged.npy"
    for split_num, split_type in enumerate(SPLIT_TYPES):
      self.averaged_data[split_type] = {}
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Loading Averaged Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        d = np.load(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + fileSuffD)
        k = np.load(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + fileSuffK)
        self.averaged_data[split_type][subject] = (d, k)
    print("\rFinished loading averaged data")

  def shuffleData(self):
    self.shuffled_data = {}
    for split_num, split_type in enumerate(SPLIT_TYPES):
      self.shuffled_data[split_type] = {}
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Shuffling Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        p = np.random.permutation(len(self.averaged_data[split_type][subject][0]))
        self.shuffled_data[split_type][subject] = (self.averaged_data[split_type][subject][0][p], self.averaged_data[split_type][subject][1][p])
    print("\rFinished shuffling data")
  
  def saveShuffledData(self):
    if (self.dim == '3D'):
      fileBase = SHUFFLED_DIR + SUFFIX_3D_DIRS
    elif (self.dim == '2D'):
      fileBase = SHUFFLED_DIR + SUFFIX_2D_DIRS
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    for split_num, split_type in enumerate(SPLIT_TYPES):
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Saving Shuffled Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        np.save(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + "D_shuffled.npy", self.shuffled_data[split_type][subject][0])
        np.save(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + "K_shuffled.npy", self.shuffled_data[split_type][subject][1])
    print("\rFinished saving shuffled data")

  def loadShuffledData(self, mode = None):
    if (mode == None):
      mode = self.dim
    if (mode == '3D'):
      fileBase = SHUFFLED_DIR + SUFFIX_3D_DIRS
    elif (mode == '2D'):
      fileBase = SHUFFLED_DIR + SUFFIX_2D_DIRS
    else:
      raise Exception("Invalid Data Dimension, please use '3D' or '2D'.")
    self.shuffled_data = {}
    fileSuffD = "D_shuffled.npy"
    fileSuffK = "K_shuffled.npy"
    for split_num, split_type in enumerate(SPLIT_TYPES):
      self.shuffled_data[split_type] = {}
      for subject in range(1, NUM_SUBJECTS+1):
        self.__liveProgress("Loading Shuffled Data", (split_num)*NUM_SUBJECTS + subject, NUM_SUBJECTS*len(SPLIT_TYPES))
        d = np.load(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + fileSuffD)
        k = np.load(fileBase + SPLIT_DIR_DICT[split_type] + "A0" + str(subject) + fileSuffK)
        self.shuffled_data[split_type][subject] = (d, k)
    print("\rFinished loading shuffled data")

  def getProcessedData(self, split_type, subject):
    return self.processed_data[split_type][subject][0], self.processed_data[split_type][subject][1]

  def getCroppedData(self, split_type, subject):
    return self.cropped_data[split_type][subject][0], self.cropped_data[split_type][subject][1]

  def getAveragedData(self, split_type, subject):  
    return self.averaged_data[split_type][subject][0], self.averaged_data[split_type][subject][1]

  def getShuffledData(self, split_type, subject):
    return self.shuffled_data[split_type][subject][0], self.shuffled_data[split_type][subject][1]  

Process Data


In [ ]:
processor = DataProcessing(mode=MODE)

processor.processData()

processor.saveProcessedData()

del(processor)

Found 9 Training Files
Found 9 Evaluation Files
Processing Data - [=-----------------] - 1/18 - 5.56%Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (1.652 sec)

Processing Data - [==----------------] - 2/18 - 11.11%Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband rippl

In [ ]:
processor = DataProcessing(mode=MODE)

processor.loadProcessedData(mode=MODE)

processor.cropData()

processor.saveCroppedData()

del(processor)

Found 9 Training Files
Found 9 Evaluation Files
Finished loading processed data
Finished cropping data
Finished saving cropped data


In [ ]:
processor = DataProcessing(mode=MODE)

processor.loadCroppedData(mode=MODE)

processor.averageData()

processor.saveAveragedData()

del(processor)

Found 9 Training Files
Found 9 Evaluation Files
Finished loading cropped data
Finished averaging data
Finished saving averaged data


In [ ]:
processor = DataProcessing(mode=MODE)

processor.loadAveragedData(mode=MODE)

processor.shuffleData()

processor.saveShuffledData()

del(processor)

Found 9 Training Files
Found 9 Evaluation Files
Finished loading averaged data
Finished shuffling data
Finished saving shuffled data


In [ ]:
processor = DataProcessing(mode=MODE)
processor.loadProcessedData(mode=MODE)
processor.loadCroppedData(mode=MODE)
processor.loadAveragedData(mode=MODE)
processor.loadShuffledData(mode=MODE)

Found 9 Training Files
Found 9 Evaluation Files
Finished loading processed data
Finished loading cropped data
Loading Averaged Data - [===============------------] - 15/27 - 55.56%

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

def plot_label_distribution(Y):
  counts = sum(Y)
  x_labels = [str(val) for val in range(NUM_CLASSES)]

  plt.bar(x_labels, counts)  # arguments are passed to plt.bar
  plt.title("Y Label Distribution")
  plt.show()

# Inspecting Data

## Inspecting Processed Data

In [ ]:
d, k = processor.getProcessedData('val', 1)

print("X Data Shape: ")
print(d.shape)
print("Y Data Shape: ")
print(k.shape)
print("Label Distribution: ")
print(sum(k))

plot_label_distribution(k)

N = 5
print("First {} Labels".format(N))
print(k[:N])

## Inspecting Cropped Data

In [ ]:
d, k = processor.getCroppedData('val', 1)

print("X Data Shape: ")
print(d.shape)
print("Y Data Shape: ")
print(k.shape)
print("Label Distribution: ")
print(sum(k))

plot_label_distribution(k)

N = 5
print("First {} Labels".format(N))
print(k[:N])

## Inspecting Averaged Data

In [ ]:
d, k = processor.getAveragedData('train', 1)

print("X Data Shape: ")
print(d.shape)
print("Y Data Shape: ")
print(k.shape)
print("Label Distribution: ")
print(sum(k))

plot_label_distribution(k)

N = 5
print("First {} Labels".format(N))
print(k[:N])

## Inspecting Shuffled Data

In [ ]:
d, k = processor.getShuffledData('train', 1)

print("X Data Shape: ")
print(d.shape)
print("Y Data Shape: ")
print(k.shape)
print("Label Distribution: ")
print(sum(k))

plot_label_distribution(k)

N = 5
print("First {} Labels".format(N))
print(k[:N])